In [9]:
import dagshub
import shap 
import mlflow
import os
import time
import csv
import json
import plotly.express as px
import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score,precision_score,recall_score,f1_score,accuracy_score,log_loss


sns.set_theme(rc={'figure.figsize':(25,25)})

In [10]:
os.environ['MLFLOW_TRACKING_USERNAME'] = 'tnbmarketplace'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd7c1a451bfc46bf96fbbe5d723becf3955dfa2b3'
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/tnbmarketplace/instacart_basket_analysis_exp_tracking.mlflow'
!dagshub login --token "d7c1a451bfc46bf96fbbe5d723becf3955dfa2b3"

2024-10-29 03:59:56,413 - [WARNING]  - [tokens.py:86] - The added token already exists in the token cache, skipping
✅ Token added successfully


In [11]:
dagshub.init(repo_name = "instacart_basket_analysis_exp_tracking",repo_owner = 'tnbmarketplace')

Initialized MLflow to track repo "tnbmarketplace/instacart_basket_analysis_exp_tracking"

Repository tnbmarketplace/instacart_basket_analysis_exp_tracking initialized!

In [12]:
from pyngrok import ngrok

In [13]:
!ngrok authtoken "2kvaYw5ZiG5bL8iM8YJBVJPk1Ru_3C16mMgmpKEBYb28PPLUe"
# Start Ngrok tunnel on H2O's port

ngrok_tunnel = ngrok.connect(8020)
print("H2O UI accessible at:", ngrok_tunnel)

In [14]:
def get_time(start):
    return time.time() - start

In [18]:
dataset_version = "1.2" # dataset . split method
model_version = "1.1.1" # algorithm . param version . used dataset 
read_as_lgb_dataset = False
train_lgb = False

In [17]:
train_features_name = []
with open("/kaggle/input/instacart-dataset-transformer-cv2/train_set_columns.txt","r") as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        train_features_name = row
train_features_name.remove('') 

test_features_name = []
with open("/kaggle/input/instacart-dataset-transformer-cv2/test_set_columns.txt","r") as f:
    csv_reader = csv.reader(f)
    for row in csv_reader:
        test_features_name = row
test_features_name.remove('')  

train_label_index = train_features_name.index('reordered')
test_label_index = test_features_name.index('reordered')

for i in range(len(test_features_name)):

    if test_features_name[i] == 'time_mean_dow_count':
        test_features_name[i] ='total_ord_count_p_dow'

    if test_features_name[i] == 'time_mean_ohod_count':
        test_features_name[i] ='total_ord_count_p_ohod'


In [19]:
if read_as_lgb_dataset == True:

    train_features_name.remove('reordered')
    test_features_name.remove('reordered')
    
    dtrain_2 = lgb.Dataset("/kaggle/input/instacart-dataset-transformer-cv2/final_prior_train_set.csv/part-00000-444aa52f-9c41-4a07-8342-7f82d9fcf6ac-c000.csv",params={'label_column':train_label_index},free_raw_data=False).construct()
    dtest_2 = lgb.Dataset("/kaggle/input/instacart-dataset-transformer-cv2/featured_test_set.csv/part-00000-017a6365-a37c-4ce4-8bf4-96e8daa33570-c000.csv",params={'label_column':test_label_index},free_raw_data=False).construct()


In [20]:
polite_crab_path = "mlflow-artifacts:/22120ced50b14bfca4ea39771cd1cdf4/9c4e42ada10b4baa956fc18db5cf8919/artifacts/lgb_gbm_model"
omniscent_hawk_path = "mlflow-artifacts:/22120ced50b14bfca4ea39771cd1cdf4/56fa0a610f224c5181cf849f812dec88/artifacts/lgb_gbm_model"
gregarious_snail_path = "mlflow-artifacts:/22120ced50b14bfca4ea39771cd1cdf4/26ad2736d34341efb4ca5402d87f01bb/artifacts/lgb_gbm_model"
likable_dog_path = "mlflow-artifacts:/c872cb0ab0ca48d481a71a9bcd2a2102/626567462ef746f8b45a4255ff51c5d1/artifacts/xgb_gbm_model"

In [21]:
dtest_2 = pd.read_csv("/kaggle/input/instacart-dataset-transformer-cv2/featured_test_set.csv/part-00000-017a6365-a37c-4ce4-8bf4-96e8daa33570-c000.csv")
dtest_2.columns = test_features_name

In [23]:
lgb_gbm1 = mlflow.lightgbm.load_model("mlflow-artifacts:/22120ced50b14bfca4ea39771cd1cdf4/631dfc7013c4449e8104de7c14ad0dce/artifacts/lgb_gbm_model")

In [49]:
dtest_2.columns

Index(['user_id', 'product_id', 'reordered', 'total_ord_count_p_dow',
       'total_ord_count_p_ohod', 'frequency_of_reorder',
       'count_of_asian_org_items', 'mean_of_asian_org_items',
       'max_count_of_products', 'min_count_of_products',
       'mean_count_of_products', 'count_ord_no_prev_purchased_items',
       'mean_ord_no_prev_purchased_items', 'product_mean_of_position',
       'number_of_user_purchased_item', 'number_of_product_co_occurred',
       'mean_of_co_ocuured_product_per_order',
       'min_of_co_ocuured_product_per_order',
       'max_of_co_ocuured_product_per_order', 'Total_streak_of_this_product',
       'mean_of_streaks_of_this_product', 'min_of_streaks_of_this_product',
       'max_of_streaks_of_this_product', 'prob_of_reordered_5',
       'prob_of_reordered_3', 'prob_of_reordered_2',
       'distrib_count_of_dow_0_p_prod', 'distrib_count_of_dow_1_p_prod',
       'distrib_count_of_dow_2_p_prod', 'distrib_count_of_dow_3_p_prod',
       'distrib_count_of_dow_4

In [61]:
xgb_gbm1.get_score(importance_type = 'total_gain')

{'product_id': 684.5582275390625,
 'frequency_of_reorder': 394991.5,
 'count_of_asian_org_items': 398055.53125,
 'mean_of_asian_org_items': 20502.474609375,
 'max_count_of_products': 58576.5703125,
 'min_count_of_products': 24146.72265625,
 'mean_count_of_products': 37715.28125,
 'count_ord_no_prev_purchased_items': 22370.37109375,
 'mean_ord_no_prev_purchased_items': 95856.8046875,
 'product_mean_of_position': 84296.046875,
 'number_of_user_purchased_item': 92388.9140625,
 'number_of_product_co_occurred': 4408.1689453125,
 'mean_of_co_ocuured_product_per_order': 20281.125,
 'min_of_co_ocuured_product_per_order': 55319.6796875,
 'max_of_co_ocuured_product_per_order': 53682.8515625,
 'Total_streak_of_this_product': 32620.439453125,
 'max_of_streaks_of_this_product': 58681.42578125,
 'prob_of_reordered_5': 185443.75,
 'prob_of_reordered_3': 66442.6953125,
 'distrib_count_of_dow_0_p_prod': 6965.7783203125,
 'distrib_count_of_dow_1_p_prod': 68158.6640625,
 'distrib_count_of_dow_2_p_prod': 

In [48]:
for col in dtest_2.columns:
    if col not in xgb_gbm1.get_score(importance_type = 'gain').keys():
        print(col)

user_id
reordered
total_ord_count_p_dow
total_ord_count_p_ohod
mean_of_streaks_of_this_product
min_of_streaks_of_this_product
prob_of_reordered_2
num_of_prod_co_ocrd_p_usr_p_prod


In [58]:
for i in lgb_gbm1.feature_importance(importance_type='gain'):
    if i > 0:
        print(i)

100.31929779052734
101.6422004699707
1927.5593013763428
86.3488998413086
129.12360191345215
111.44389915466309
30.091100692749023
79915566.78660393
235.76710510253906


In [57]:
px.bar(x=lgb_gbm1.feature_importance(importance_type='gain'),y=dtest_2.drop(["reordered"],axis = 1).columns,orientation='h')

In [40]:
lgb_gbm1.feature_importance(importance_type='gain').shape

(36,)

In [19]:
preds = lgb_gbm1.predict(dtest_2.drop("reordered",axis=1))
pred_digits = (preds > 0.5).astype(int)
tp_indices = (pred_digits == 1 ) & ( dtest_2['reordered'] == 1)
fp_indices = (pred_digits == 1 ) & ( dtest_2['reordered'] == 0)
tn_indices = (pred_digits == 0 ) & ( dtest_2['reordered'] == 0)
fn_indices = (pred_digits == 0 ) & ( dtest_2['reordered'] == 1)


In [20]:
# # shap analysis
# shap.initjs()
# shap_values = shap.explainers.TreeExplainer(lgb_gbm1,dtest_2.drop("reordered",axis=1)) \
#                     .shap_values(dtest_2.drop("reordered",axis=1))
# shap_values_array = np.array(shap_values)


In [21]:
# shap.summary_plot(shap_values[1],dtest_2.drop("reordered",axis=1),show=False)
# plt.savefig('shap_summary_of_lgb.png')

In [22]:
# shap.plots.violin(shap_values_array,feature_names=test_features_name)

In [23]:
from shapash import SmartExplainer

In [28]:
feature_names_dict = {
    'user_id': 'User ID',
    'product_id': 'Product ID',
    'reordered': 'Reordered Status',
    'total_ord_count_p_dow': 'Total Order Count per Day of Week',
    'total_ord_count_p_ohod': 'Total Order Count per Order Hour of Day',
    'frequency_of_reorder': 'Frequency of Reorder',
    'count_of_asian_org_items': 'Count of Asian Organic Items',
    'mean_of_asian_org_items': 'Mean Count of Asian Organic Items',
    'max_count_of_products': 'Maximum Count of Products per Order',
    'min_count_of_products': 'Minimum Count of Products per Order',
    'mean_count_of_products': 'Mean Count of Products per Order',
    'count_ord_no_prev_purchased_items': 'Order Count of Non-Previously Purchased Items',
    'mean_ord_no_prev_purchased_items': 'Mean Order Count of Non-Previously Purchased Items',
    'product_mean_of_position': 'Mean Position of Product in Orders',
    'number_of_user_purchased_item': 'Number of Users who Purchased Item',
    'number_of_product_co_occurred': 'Number of Co-Occurring Products',
    'mean_of_co_ocuured_product_per_order': 'Mean of Co-Occurring Products per Order',
    'min_of_co_ocuured_product_per_order': 'Minimum of Co-Occurring Products per Order',
    'max_of_co_ocuured_product_per_order': 'Maximum of Co-Occurring Products per Order',
    'Total_streak_of_this_product': 'Total Streak of This Product',
    'mean_of_streaks_of_this_product': 'Mean Streaks of This Product',
    'min_of_streaks_of_this_product': 'Minimum Streaks of This Product',
    'max_of_streaks_of_this_product': 'Maximum Streaks of This Product',
    'prob_of_reordered_5': 'Probability of Reorder within next 5 orders',
    'prob_of_reordered_3': 'Probability of Reorder within next 3 orders',
    'prob_of_reordered_2': 'Probability of Reorder within next 2 orders',
    'distrib_count_of_dow_0_p_prod': 'Distribution Count of Product on Day 0',
    'distrib_count_of_dow_1_p_prod': 'Distribution Count of Product on Day 1',
    'distrib_count_of_dow_2_p_prod': 'Distribution Count of Product on Day 2',
    'distrib_count_of_dow_3_p_prod': 'Distribution Count of Product on Day 3',
    'distrib_count_of_dow_4_p_prod': 'Distribution Count of Product on Day 4',
    'distrib_count_of_dow_5_p_prod': 'Distribution Count of Product on Day 5',
    'distrib_count_of_dow_6_p_prod': 'Distribution Count of Product on Day 6',
    'prob_of_being_reordered': 'Probability of Being Reordered',
    'num_of_ord_purch_p_prod': 'Number of Orders Purchasing Product',
    'prod_mean_of_position_p_user': 'Mean Position of Product per User',
    'num_of_prod_co_ocrd_p_usr_p_prod': 'Number of Product Co-Occurrences per User per Product'
}


34

In [25]:
xpl = SmartExplainer(
    features_dict = feature_names_dict,
    model = lgb_gbm1
)

In [26]:
xpl.compile(
    x = dtest_2.drop('reordered',axis=1),
    y_pred = pd.DataFrame(preds,columns=['predictions']),
    y_target = dtest_2['reordered']
)

INFO: Shap explainer type - <shap.explainers._tree.TreeExplainer object at 0x79af420e8820>


In [31]:
xpl.plot.features_importance()

In [32]:
xpl.plot.top_interactions_plot(nb_top_interactions=5)

In [ ]:
fig = xpl.plot.contribution_plot(col='Age')